# Dependencies

In [62]:
pacman::p_load(dplyr)

# Functions

In [63]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")

# Paths

In [64]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [65]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

# Main

bind all metadata

In [66]:
meta_div <- readr::read_rds(file = paste0(datapath,"capTCRseq/meta_div_goodsamples.rds"))

In [67]:
table(meta_div$cycle)


X00 X01 X02 X03 X04 X05 
 42 150 164 160 147 147 

In [68]:
allAdaptive <- readr::read_rds(file = paste0(datapath, "Adaptivedatasets/allAdaptive_good.rds"))

In [69]:
# remove those with no TCR data
meta_div <- meta_div[ !is.na(meta_div$file),]

In [70]:
table(meta_div$cycle, meta_div$sampletype)

     
      PBMC Tumor cfDNA
  X00   24    17     0
  X01   47     0    56
  X02   53     0    64
  X03   52     0    56
  X04   42     0    51
  X05   40     0    43

In [71]:
meta_div$study <- "INTERCEPT"
meta_div$Agegroup <- "Child"
meta_div$Sex <- NA
meta_div$disease <- meta_div$cancergroup
meta_div$sample_name <- meta_div$basename

In [72]:
allAdaptive$sampletype <- "PBMC"
#allAdaptive$sampletype[grepl("Cord", allAdaptive$sample_name )] <- "Cord blood"

In [73]:
# replace age with Age in allAdative
colnames(allAdaptive)[colnames(allAdaptive) == "age"] <- "Age"
# subject in allAdaptive is the same as Patient in meta_div
colnames(allAdaptive)[colnames(allAdaptive) == "subject"] <- "Patient"

In [74]:
mycols <- intersect(colnames(meta_div), colnames(allAdaptive))

In [75]:
mycols

[1] "Patient"     "Age"         "sampletype"  "study"       "Agegroup"   
[6] "Sex"         "disease"     "sample_name"

In [76]:
all_metadata <- rbind(allAdaptive[,mycols], meta_div[,mycols])

In [77]:
dim(all_metadata)

[1] 1677    8

In [78]:
head(all_metadata)

,Patient,Age,sampletype,study,Agegroup,Sex,disease,sample_name
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
21,Peds_Ctrl_2,NA,PBMC,Henderson,Child,NA,Peds healthy Control,Peds_Ctrl_2
22,MAS_3,NA,PBMC,Henderson,Child,NA,MAS,MAS_3
23,Peds_Febrile_Ctrl_3,NA,PBMC,Henderson,Child,NA,Peds febrile Control,Peds_Febrile_Ctrl_3
25,KD_4,NA,PBMC,Henderson,Child,NA,KD,KD_4
26,KD_3,NA,PBMC,Henderson,Child,NA,KD,KD_3
29,sJIA_7,NA,PBMC,Henderson,Child,NA,sJIA,sJIA_7


In [79]:
table(all_metadata$study)


        Emerson       Henderson       INTERCEPT        Mitchell Mitchell_Denver 
            773              25             545             203             131 

In [80]:
#save all_metadata
readr::write_rds(all_metadata, file = paste0(datapath, "all_metadata_good.rds"))

In [81]:
table(all_metadata$Agegroup)


 Adolescent       Adult       Child     Neonate      Senior Young Adult 
         98         505         788          35          27          89 

## All PBMC data

In [82]:
# remove cfDNA samples
all_metadata_pbmc <- all_metadata[!grepl("cfDNA", all_metadata$sample_name),]

all_metadata_pbmc <- all_metadata_pbmc[!grepl("FFPE", all_metadata_pbmc$sample_name),]

In [83]:
dim(all_metadata_pbmc)

[1] 1390    8

In [84]:
table(all_metadata_pbmc$study)


        Emerson       Henderson       INTERCEPT        Mitchell Mitchell_Denver 
            773              25             258             203             131 

## Discovery set of baseline only

In [85]:
discovery_metadata <- all_metadata_pbmc[ all_metadata_pbmc$study != "Mitchell_Denver",]

In [86]:
discovery_metadata <- discovery_metadata[ !grepl("Keck", discovery_metadata$sample_name),] # emerson validation set

In [87]:
dim(discovery_metadata[ discovery_metadata$study == "INTERCEPT",])

[1] 258   8

In [88]:
dim(discovery_metadata)

[1] 1140    8

#### sample intercept study

In [89]:
# subset to INTERCEPT study
intonly <- discovery_metadata[ discovery_metadata$study == "INTERCEPT",]

In [90]:
length(table(intonly$Patient)) # number of patients

[1] 112

subset to baseline only

In [91]:
intonly_01 <- intonly[ grepl( "-01",intonly$sample_name),]
normal_int <- intonly[ grepl( "Normal",intonly$disease),]
lfs_int <- intonly[ grepl( "LFS",intonly$sample_name),]

In [92]:
# rbind
intonly_01_normal_lfs <- rbind(intonly_01, normal_int, lfs_int)

In [93]:
dim(intonly_01_normal_lfs)

[1] 71  8

In [94]:
myfreq <- as.data.frame(table(intonly_01_normal_lfs$Patient, intonly_01_normal_lfs$disease), stringsAsFactors = F)

In [95]:
myfreq <- myfreq[myfreq$Freq > 0,]
(myfreq)

,Var1,Var2,Freq
,<chr>,<chr>,<int>
62,LFS_TCR_4535,LFS,1
63,LFS_TCR_4570,LFS,1
64,LFS_TCR_4602,LFS,1
65,LFS_TCR_4758,LFS,1
66,LFS_TCR_4759,LFS,1
67,LFS_TCR_4783,LFS,1
68,LFS_TCR_4837,LFS,1
69,LFS_TCR_4838,LFS,1
70,LFS_TCR_5014,LFS,1


In [96]:
# randomly select 60% of baseline samples from each disease group 
set.seed(7)
myfreq %>% group_by(Var2) %>% sample_frac(0.6) -> discovery_sample_freq

In [97]:
discovery_sample_freq

Var1,Var2,Freq
<chr>,<chr>,<int>
LFS_TCR_5067,LFS,1
LFS_TCR_4602,LFS,1
LFS_TCR_4837,LFS,1
LFS_TCR_4758,LFS,1
LFS_TCR_4570,LFS,1
LFS_TCR_4783,LFS,1
CHP_416,Leukemia,1
CHP_365,Leukemia,1
CHP_352,Leukemia,1


In [98]:
# all int patients in discovery set
int_discovery_patients <- discovery_sample_freq$Var1

In [99]:
length(int_discovery_patients)

[1] 42

In [100]:
dim(discovery_metadata)

[1] 1140    8

In [101]:
dim(discovery_metadata[ discovery_metadata$study == "INTERCEPT",])

[1] 258   8

In [102]:
all_discovery_patients <- c(discovery_metadata$Patient[ discovery_metadata$study != "INTERCEPT"], int_discovery_patients) # all patients in discovery set

In [103]:
discovery_metadata <- discovery_metadata[ discovery_metadata$Patient %in% all_discovery_patients,]

In [104]:
length(table(discovery_metadata$Patient[ discovery_metadata$study == "INTERCEPT"] ))

[1] 42

In [105]:
# CHP_407 is dupliucate sample one from lymphoma one from ffpe
dim(discovery_metadata)

[1] 973   8

In [106]:
table(discovery_metadata$study)


  Emerson Henderson INTERCEPT  Mitchell 
      654        25        91       203 

## Validation set

In [107]:
table(all_metadata_pbmc$study)


        Emerson       Henderson       INTERCEPT        Mitchell Mitchell_Denver 
            773              25             258             203             131 

In [108]:
Mitchell_Denver_metadata <- all_metadata_pbmc[ all_metadata_pbmc$study == "Mitchell_Denver",]
keck_metadata <- all_metadata_pbmc[ grepl("Keck", all_metadata_pbmc$sample_name),] # emerson validation set

In [109]:
# subset to INTERCEPT study
intonly <- all_metadata_pbmc[ all_metadata_pbmc$study == "INTERCEPT",]

In [110]:
intonly_01 <- intonly[ grepl( "-01",intonly$sample_name),]
normal_int <- intonly[ grepl( "Normal",intonly$disease),]
lfs_int <- intonly[ grepl( "LFS",intonly$sample_name),]

In [111]:
# rbind
intonly_01_normal_lfs <- rbind(intonly_01, normal_int, lfs_int)

In [112]:
head(intonly_01_normal_lfs)

,Patient,Age,sampletype,study,Agegroup,Sex,disease,sample_name
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1671,CHP_346,1,PBMC,INTERCEPT,Child,NA,Solid,CHP_346-01-PBMC-DNA
1731,CHP_347,13,PBMC,INTERCEPT,Child,NA,Leukemia,CHP_347-01-PBMC-DNA
1821,CHP_348,18,PBMC,INTERCEPT,Child,NA,Solid,CHP_348-01-PBMC-DNA
1921,CHP_349,17,PBMC,INTERCEPT,Child,NA,Leukemia,CHP_349-01-PBMC-DNA
1981,CHP_350,1,PBMC,INTERCEPT,Child,NA,Solid,CHP_350-01-PBMC-DNA
2121,CHP_352,17,PBMC,INTERCEPT,Child,NA,Leukemia,CHP_352-01-PBMC-DNA


In [113]:
int_validation_patients <- intonly_01_normal_lfs$sample_name[!intonly_01_normal_lfs$Patient %in% int_discovery_patients]

In [114]:
int_discovery_patients

[1] "LFS_TCR_5067" "LFS_TCR_4602" "LFS_TCR_4837" "LFS_TCR_4758" "LFS_TCR_4570"
 [6] "LFS_TCR_4783" "CHP_416"      "CHP_365"      "CHP_352"      "CHP_420"     
[11] "CHP_396"      "CHP_422"      "CHP_377"      "CHP_411"      "CHP_413"     
[16] "CHP_355"      "CHP_397"      "CHP_373"      "CHP_354"      "CHP_353"     
[21] "CHP_387"      "CHP_412"      "CHP_366"      "CHP_415"      "CHP_371"     
[26] "CHP_16YO_M"   "CHP_1YO_M_B"  "CHP_3YO_M_A"  "CHP_5YO_M"    "CHP_10YO_M"  
[31] "CHP_15YO_F_B" "CHP_3YO_F"    "CHP_15YO_F_A" "CHP_409"      "CHP_363"     
[36] "CHP_381"      "CHP_394"      "CHP_414"      "CHP_364"      "CHP_418"     
[41] "CHP_404"      "CHP_368"

In [115]:
length(int_validation_patients)

[1] 29

In [116]:
dim(intonly_01_normal_lfs[ intonly_01_normal_lfs$sample_name %in% int_validation_patients,])

[1] 29  8

In [117]:
validation_metadata <- rbind(keck_metadata, 
Mitchell_Denver_metadata,
intonly_01_normal_lfs[ intonly_01_normal_lfs$sample_name %in% int_validation_patients,])


In [118]:
dim(validation_metadata)

[1] 279   8

In [119]:
dim(discovery_metadata)

[1] 973   8

In [120]:
dim(all_metadata_pbmc)

[1] 1390    8

In [121]:
# save discovery and validation metadata
readr::write_rds(discovery_metadata, file = paste0(datapath, "discovery_metadata_01.rds"))
readr::write_rds(validation_metadata, file = paste0(datapath, "validation_metadata_01.rds"))